## GPU Configuration and Imports <a class="anchor" id="GPU-Configuration-and-Imports"></a>

In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')


2025-05-19 06:45:22.590718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747637122.731207    2213 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747637122.768085    2213 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747637123.050624    2213 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747637123.050661    2213 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747637123.050664    2213 computation_placer.cc:177] computation placer alr

In [2]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)

[]


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
import os
import pandas as pd
import datetime
import h5py

from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, Conv2D, LayerNormalization, SeparableConv2D, Normalization, BatchNormalization, Dense
from tensorflow.nn import relu

## **Neural Receiver <a class="anchor" id="Neural-Receiver"></a>**

In [28]:
class ResidualBlock(Model):
    def __init__(self):
        super(ResidualBlock, self).__init__()
        # self._layer_norm_1 = LayerNormalization(axis=[-1])
        self._conv_1 = SeparableConv2D(filters= 64,
                              kernel_size=[3,3],
                              padding='same',
                              activation=None)

        # self._layer_norm_2 = LayerNormalization(axis=[-1])
        self._conv_2 = SeparableConv2D(filters= 64,
                              kernel_size=[3,3],
                              padding='same',
                              activation=None)

    def call(self, inputs):
        # z = self._layer_norm_1(inputs)
        z = relu(inputs)
        z = self._conv_1(z)
        # z = self._layer_norm_2(z)
        z = relu(z)
        z = self._conv_2(z) # [batch size, num time samples, num subcarriers, num_channels]
        # Skip connection
        z = z + inputs

        return z

class CustomNeuralReceiver(Model):
    def __init__(self, num_res_block):
        super(CustomNeuralReceiver, self).__init__()
        self._num_resblock = num_res_block

        # Input convolution
        self._input_conv = SeparableConv2D(filters= 64,
                                  kernel_size=[3,3],
                                  padding='same',
                                  activation=None)
        
    
        # Residual blocks
        self._res_block = [ResidualBlock() for i in range(self._num_resblock)]
        # self._res_block_1 = ResidualBlock()
        # self._res_block_2 = ResidualBlock()
        # self._res_block_3 = ResidualBlock()
        # self._res_block_4 = ResidualBlock()
        # self._dense_1 = Dense(64)
        # self._dense_2 = Dense(64)
        # self._dense_3 = Dense(32)

        # Output conv
        self._output_conv = Conv2D(filters= 2,    # QPSK
                                   kernel_size=[3,3],
                                   padding='same',
                                   activation=None)

    def call(self, inputs):
        # Input conv
        z = self._input_conv(inputs)
        # Residual blocks
        for i in range(self._num_resblock):
            z = self._res_block[i](z)
        # z = self._res_block_1(z)
        # z = self._res_block_2(z)
        # z = self._res_block_3(z)
        # z = self._res_block_4(z)
        # z = self._dense_1(z)
        # z = relu(z)
        # z = self._dense_2(z)
        # z = relu(z)
        # z = self._dense_3(z)
        # z = relu(z)
        # Output conv
        z = self._output_conv(z)
        return z

#**Load Data**

- Function to load and preprocess data (y - receiver data and c - label data)

In [29]:
def load_hdf5(parent_name, group_name):
    with h5py.File(f'{parent_name}.hdf5', "r") as f:
        b = f[f"{group_name}_b"][:]
        c = f[f"{group_name}_c"][:]
        y = f[f"{group_name}_y"][:]
        r = f[f"{group_name}_r"][:]
    return b, c, y, r

def load_pickle(parent_name, group_name):
    """Saves data to a pickle file."""
    def load_from_pickle(filename):
        with open(filename, "rb") as f:
            return pickle.load(f)

    b = load_from_pickle(f'{parent_name}/{group_name}.b.pkl')
    c = load_from_pickle(f'{parent_name}/{group_name}.c.pkl')
    y = load_from_pickle(f'{parent_name}/{group_name}.y.pkl')
    r = load_from_pickle(f'{parent_name}/{group_name}.r.pkl')

    return b, c, y, r

def data_loader(df, dir, saved_dataset='hdf5'):
    assert saved_dataset in ['hdf5', 'pickle'], "saved data set should be 'pickle' or 'hdf5'."
    assert df['nTBSize'].nunique() == 1, "Not all elements have the same TB size."
    # assert df['Dmrs_mask'].map(len).nunique() == 1, "Not all elements have the same number of Dmrs/Data symbols."

    for pusch_record in df.itertuples():
        data_filename = pusch_record.Data_filename
        data_dirname = pusch_record.Data_dirname
        esno_db = pusch_record.Esno_db
        index = pusch_record.index
        if saved_dataset == 'hdf5':
            b,c,y, r = load_hdf5(f'{dir}/{data_dirname}', data_filename)
        else:
            b,c,y, r = load_pickle(f'{dir}/{data_dirname}', data_filename)
        yield index, esno_db, c, y, b, r

def preprocessing(index, esno_db, c, y, b, r):
    y = tf.concat([tf.math.real(y), tf.math.imag(y)], axis = 0)
    y = tf.transpose(y, perm=[2,1,0])
    y = (y - tf.reduce_mean(y)) / tf.math.reduce_std(y)
    r = tf.concat([tf.math.real(r), tf.math.imag(r)], axis = 0)
    r = tf.transpose(r, perm=[2,1,0])

    y_r = tf.concat([y, r], axis = -1)
    return index, esno_db, c, y, b, r, y_r



**Load Data into Dataset**

* **Setup dataset_dir**

In [30]:
dataset_dir = f'/content/'
pickles_dir = f'{dataset_dir}/pickle'
hdf5_dir = f'{dataset_dir}/hdf5'
parquet_dir = f'{dataset_dir}/parquet'

parquet_name = '4RB_dataset_186k_samples' #4RB

* **Read and split data_folder into train and test set**

* Read and load data into DataFrame, then split them into 3 group (SNR -low,mid,high)

In [31]:
df = pd.read_parquet(f'{parquet_dir}/{parquet_name}.parquet', engine="pyarrow")
df = df.reset_index()

split_rate = 0.95

df_low = df[df['Esno_db'] <= -5]
df_mid = df[(df['Esno_db'] <= 0) & (df['Esno_db'] > -5)]
df_high = df[df['Esno_db'] > -0]

NUM_SAMPLE_LOW = len(df_low)
NUM_SAMPLE_MID = len(df_mid)
NUM_SAMPLE_HIGH = len(df_high)

df_low = df_low.sample(frac=1)
df_mid = df_mid.sample(frac=1)
df_high = df_high.sample(frac=1)

test_df_low = df_low.iloc[int(NUM_SAMPLE_LOW*split_rate):]
train_df_low = df_low.iloc[:int(NUM_SAMPLE_LOW*split_rate)]

test_df_mid = df_mid.iloc[int(NUM_SAMPLE_MID*split_rate):]
train_df_mid = df_mid.iloc[:int(NUM_SAMPLE_MID*split_rate)]

test_df_high = df_high.iloc[int(NUM_SAMPLE_HIGH*split_rate):]
train_df_high = df_high.iloc[:int(NUM_SAMPLE_HIGH*split_rate)]

train_df = pd.concat([train_df_low, train_df_mid, train_df_high]).reset_index(drop=True).reset_index()

print(f"Low: Total Size: {len(df_low)} sample, Test Size: {len(test_df_low)} sample, Train Size: {len(train_df_low)} sample")
print(f"Mid: Total Size: {len(df_mid)} sample, Test Size: {len(test_df_mid)} sample, Train Size: {len(train_df_mid)} sample")
print(f"High: Total Size: {len(df_high)} sample, Test Size: {len(test_df_high)} sample, Train Size: {len(train_df_high)} sample")


ImportError: Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.

* **Load data into Dataset object**

In [32]:
def load_data(train_df, test_df, hdf5_dir, batch_train, batch_test):
  train_set = tf.data.Dataset.from_generator(
            lambda: data_loader(train_df, hdf5_dir),
            output_types=(tf.int32, tf.float32, tf.float32, tf.complex64, tf.float32, tf.complex64))

  train_set = train_set.cache()
  train_set = train_set.prefetch(tf.data.AUTOTUNE)
  train_set = train_set.map(preprocessing).batch(batch_train)

  # Load test_data into Dataset
  test_set = tf.data.Dataset.from_generator(
              lambda: data_loader(test_df, hdf5_dir),
              output_types=(tf.int32, tf.float32, tf.float32, tf.complex64, tf.float32, tf.complex64))

  test_set = test_set.cache()
  test_set = test_set.prefetch(tf.data.AUTOTUNE)
  test_set = test_set.map(preprocessing).batch(batch_test)
  return train_set, test_set


In [33]:
BATCH_TRAIN = 256
BATCH_TEST = 64

train_set, test_set_low = load_data(train_df, test_df_low, hdf5_dir, BATCH_TRAIN, BATCH_TEST)
_, test_set_mid = load_data(train_df_mid, test_df_mid, hdf5_dir, BATCH_TRAIN, BATCH_TEST)
_, test_set_high = load_data(train_df_high, test_df_high, hdf5_dir, BATCH_TRAIN, BATCH_TEST)



NameError: name 'train_df' is not defined

In [34]:
# First load data for accelerating
for n, (index, esno_db, c, y, b, r, y_r) in enumerate(test_set_low):
    print(y.shape)

for n, (index, esno_db, c, y, b, r, y_r) in enumerate(test_set_mid):
    print(y.shape)

for n, (index, esno_db, c, y, b, r, y_r) in enumerate(test_set_high):
    print(y.shape)

for n, (index, esno_db, c, y, b, r, y_r) in enumerate(train_set):
    print(y.shape)


NameError: name 'test_set_low' is not defined

- Checking size of a sample data in Dataset
- Preprocessing is called when load Data

#**Training Model**



*   Creat Model and Optimizer Instance
*   Set mode of "pretrained" ( True to load pretrained weights if exist )

In [35]:
NUM_ANTENNA_8RX = 18
NUM_ANTENNA_4RX = 10

In [36]:
model = CustomNeuralReceiver(10)
inputs = tf.zeros([1,48,14,NUM_ANTENNA_4RX])
model(inputs)
model.summary()

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

pretrained = False

Model: "custom_neural_receiver_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ separable_conv2d_56             │ (1, 48, 14, 64)        │           794 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_26               │ ?                      │         9,472 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_27               │ ?                      │         9,472 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_28               │ ?                      │         9,472 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_29               │ ?                      │         9,472 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_30               │ ?                      │         9,472 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_31               │ ?                      │         9,472 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_32               │ ?                      │         9,472 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_33               │ ?                      │         9,472 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_34               │ ?                      │         9,472 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_35               │ ?                      │         9,472 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (1, 48, 14, 2)         │         1,154 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 96,668 (377.61 KB)

 Trainable params: 96,668 (377.61 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
import openvino as ov

In [38]:
ov_model = ov.convert_model(model, input=[-1,48,14,10])
ov.save_model(ov_model, 'VHT_96k_model.xml')

In [193]:
core = Core()

# Load model
model_path = "new_model.xml"  # hoặc .onnx, .bin,...
model = core.read_model(model=model_path)
model.reshape([2,48,14,10])
ov.save_model(model, 'new_model_1.xml')

In [182]:
from openvino.runtime import Core
import numpy as np
import time
# Khởi tạo OpenVINO runtime
core = Core()

# Load model
model_path = "new_model.xml"  # hoặc .onnx, .bin,...
model = core.read_model(model=model_path)

# Compile model
compiled_model = core.compile_model(model=model, device_name="CPU")

# Tạo 1 Infer Request
infer_request = compiled_model.create_infer_request()

# Chuẩn bị input
input_layer = compiled_model.input(0)
dummy_input = np.random.randn(40, 48, 14, 10).astype(np.float32)
infer_request.infer({input_layer: dummy_input})
dummy_input = np.random.randn(40, 48, 14, 10).astype(np.float32)
start = time.time()
# Gửi request infer
infer_request.infer({input_layer: dummy_input})

# Lấy output
output_layer = compiled_model.output(0)
result = infer_request.get_output_tensor(output_layer.index).data
print((time.time()-start)*1000)
print("Output shape:", result.shape)


33.57076644897461
Output shape: (40, 48, 14, 2)


In [ ]:
# Define our metrics
train_batch_loss = tf.keras.metrics.Mean('Epoch_train_loss', dtype=tf.float32)
val_batch_loss_low = tf.keras.metrics.Mean('Epoch_val_loss_low', dtype=tf.float32)
val_batch_loss_mid = tf.keras.metrics.Mean('Epoch_val_loss_mid', dtype=tf.float32)
val_batch_loss_high = tf.keras.metrics.Mean('Epoch_val_loss_high', dtype=tf.float32)


- **Define Train_step and Loss_cal**





In [ ]:
def loss_cal(pred, labels):
  bce = tf.nn.sigmoid_cross_entropy_with_logits(labels, pred)
  bce = tf.reduce_mean(bce)
  loss = bce
  return loss


#train model with nRB
@tf.function
def train_step(inputs, labels):
  with tf.GradientTape() as tape:
    llr = model(inputs)

    # Convert output size to label size (Remove DMRS, Demapper)
    llr = tf.concat([llr[...,0:3,:],llr[...,4:11,:], llr[...,12:14,:]],axis=-2)
    llr = tf.transpose(llr, perm=[0,2,1,3])
    llr = tf.reshape(llr, [llr.shape[0],(llr.shape[1]*llr.shape[2]*llr.shape[3])])

    # Loss calculation
    loss = loss_cal(llr, labels)

  # Apply Gradient Descent
  weights = model.trainable_weights
  grads = tape.gradient(loss, weights)
  optimizer.apply_gradients(zip(grads, weights))

  return loss

#test model with nRB
@tf.function
def val_step(inputs, labels):
  llr = model(inputs)
  llr = tf.concat([llr[...,0:3,:],llr[...,4:11,:], llr[...,12:14,:]],axis=-2)
  llr = tf.transpose(llr, perm=[0,2,1,3])
  llr = tf.reshape(llr, [llr.shape[0],(llr.shape[1]*llr.shape[2]*llr.shape[3])])
  loss = loss_cal(llr, labels)
  return loss

- **Load and save pretrained Model**

In [20]:
def load_weights(model, pretrained_weights_path):
    # Build Model with random input
    # Load weights
  with open(pretrained_weights_path, 'rb') as f:
    weights = pickle.load(f)
    model.set_weights(weights)
    print(f"Loaded pretrained weights from {pretrained_weights_path}")

def save_weights(model, model_folder_path, model_file_name):
    # Save the weights in a file
    model_weights_path = os.path.join(model_folder_path, model_file_name)
    weights = model.get_weights()
    with open(model_weights_path, 'wb') as f:
        pickle.dump(weights, f)

def save_trainable_weights(model, model_folder_path, model_file_name):
    # Save the weights in a file
    model_weights_path = os.path.join(model_folder_path, model_file_name)
    weights = model.trainable_weights
    with open(model_weights_path, 'wb') as f:
        pickle.dump(weights, f)

* Load pretrained weight if existed ?

In [ ]:
if pretrained:
  pretrained_weights_path = '/content/drive/MyDrive/AI_for_PUSCH/VHT_neural_receiver/weight_4RB_normalize_with_DMRS_org.pkl'
  load_weights(model, pretrained_weights_path)

- **Training model**

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
#Initialize folder log for tensorboard
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

#Setup log_path
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)

val_log_dir = 'logs/gradient_tape/' + current_time + '/val'
val_summary_writer = tf.summary.create_file_writer(val_log_dir)

* ***If using VSCode, run "tensorboard --logdir /log_path" in terminal***

In [ ]:
%tensorboard --logdir logs/gradient_tape/



*   **Define global params for training**



In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
# Define num EPOCHS, Model_path, Model_name to save model during training
EPOCHS = 1000
SAVE_AFTER_NUM_EPOCH = 2
model_folder_path = "VHT_neural_receiver/"    #Custom here

In [ ]:
if not pretrained:
  pre_batch_loss = 1e9
  for epoch in range(EPOCHS):
      start_time = time.time()
      batch_loss = np.array([])
      batch_val_loss_low = np.array([])
      batch_val_loss_mid = np.array([])
      batch_val_loss_high = np.array([])
      print(
        f'Epoch {epoch}, '
      )

    # Load batch data for 4RB training
      for iter, (index, esno_db, c, y, b,r, y_r) in enumerate(train_set):
          # calculate loss and optimize
          loss = train_step(y_r, c)
          batch_loss = np.append(batch_loss, loss)


      # Load batch data for 4RB validating
      for iter, (index, esno_db, c, y, b,r, y_r) in enumerate(test_set_low):
          loss_val = val_step(y_r, c)
          batch_val_loss_low = np.append(batch_val_loss_low, loss_val)

      for iter, (index, esno_db, c, y, b,r, y_r) in enumerate(test_set_mid):
          loss_val = val_step(y_r, c)
          batch_val_loss_mid = np.append(batch_val_loss_mid, loss_val)

      for iter, (index, esno_db, c, y, b,r, y_r) in enumerate(test_set_high):
          loss_val = val_step(y_r, c)
          batch_val_loss_high = np.append(batch_val_loss_high, loss_val)


      # Write batch loss to log file
      train_batch_loss(tf.reduce_mean(batch_loss))
      with train_summary_writer.as_default():
        tf.summary.scalar('BCE_Epoch_Loss', train_batch_loss.result(), step=epoch+134)

      val_batch_loss_low(tf.reduce_mean(batch_val_loss_low))
      with val_summary_writer.as_default():
        tf.summary.scalar('BCE_Epoch_Loss', val_batch_loss_low.result(), step=epoch+134)

      val_batch_loss_mid(tf.reduce_mean(batch_val_loss_mid))
      with val_summary_writer.as_default():
        tf.summary.scalar('BCE_Epoch_Loss', val_batch_loss_mid.result(), step=epoch+134)

      val_batch_loss_high(tf.reduce_mean(batch_val_loss_high))
      with val_summary_writer.as_default():
        tf.summary.scalar('BCE_Epoch_Loss', val_batch_loss_high.result(), step=epoch+134)

      time_taken = time.time() - start_time

      # Log print
      print(
          f'Epoch {epoch}, '
          f'Train_Loss: {train_batch_loss.result():0.4f}, '
          f'Val_Loss_low: {val_batch_loss_low.result():0.4f}, '
          f'Val_Loss_mid: {val_batch_loss_mid.result():0.4f}, '
          f'Val_Loss_high: {val_batch_loss_high.result():0.4f}, '
          f'Time taken: {time_taken:0.2f}'
      )
      
      # Update weight during training if (high/mid/low) val decrease (Could cmt)
      # if pre_batch_loss >= val_batch_loss_high.result():
      #   pre_batch_loss = val_batch_loss_high.result()
      #   model_file_name = f"weight_4RB_high_SNR_dynamic_config.pkl"
      #   save_weights(model, model_folder_path, model_file_name)
      #   print(
      #       f'Save model at epoch {epoch}'
      #   )

      train_batch_loss.reset_state()
      val_batch_loss_low.reset_state()
      val_batch_loss_mid.reset_state()
      val_batch_loss_high.reset_state()


* **Save weight**

In [ ]:
    # Save the weights in a file
model_folder_path = "VHT_neural_receiver/"
model_file_name = "weight_name.pkl"
save_weights(model, model_folder_path, model_file_name)